## Approach 2
   - For Feedback prize-2021 competition, approach 2 will be divide each content into batch of 512, in that way we won't need to trim the content till 512 .
   - https://towardsdatascience.com/how-to-apply-transformers-to-any-length-of-text-a5601410af7f - Followed this blog fir this approach

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Load the model**

In [2]:
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
import pprint as pprint
label_to_idx = {'B-Claim': 0, 'I-Claim': 1,
                'B-Evidence': 2, 'I-Evidence': 3,
                'B-Position': 4, 'I-Position': 5,
                'B-Concluding Statement': 6, 'I-Concluding Statement': 7,
                'B-Lead': 8, 'I-Lead': 9,
                'B-Counterclaim': 10, 'I-Counterclaim': 11,
                'B-Rebuttal': 12, 'I-Rebuttal': 13,
                'O': 14}
idx_to_label = {v:k for k,v in label_to_idx.items()}
idx_to_label

{0: 'B-Claim',
 1: 'I-Claim',
 2: 'B-Evidence',
 3: 'I-Evidence',
 4: 'B-Position',
 5: 'I-Position',
 6: 'B-Concluding Statement',
 7: 'I-Concluding Statement',
 8: 'B-Lead',
 9: 'I-Lead',
 10: 'B-Counterclaim',
 11: 'I-Counterclaim',
 12: 'B-Rebuttal',
 13: 'I-Rebuttal',
 14: 'O'}

In [5]:
MAX_LEN = 512
tokenizer = BertTokenizerFast.from_pretrained('../input/feedback-nb4/bert-base-uncased-tokenizer')

In [6]:
model = BertForTokenClassification.from_pretrained('../input/feedback-nb4/feedback-bert-uncased-model2')
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [7]:
def model_pred(content):
    sentence = content.strip().split()
    # Step1 - Call the tokenizer to encode
    encoding = tokenizer.encode_plus(sentence,
                                     add_special_tokens=False,
                                     return_tensors="pt",
                                     is_split_into_words=True,
                                     return_offsets_mapping=True)
    
    # Step2 - divide it into chunks
    chunksize = 512
    input_id_chunks = list(encoding['input_ids'][0].split(chunksize - 2))
    mask_chunks = list(encoding['attention_mask'][0].split(chunksize - 2))
    offset_mapping_chunks = list(encoding['offset_mapping'][0].split(chunksize - 2))
    
    # Step3: loop through each chunk
    for i in range(len(input_id_chunks)):
        # add CLS and SEP tokens to input IDs
        input_id_chunks[i] = torch.cat([
            torch.tensor([101]), input_id_chunks[i], torch.tensor([102])
        ])
        # add attention tokens to attention mask
        mask_chunks[i] = torch.cat([
            torch.tensor([1]), mask_chunks[i], torch.tensor([1])
        ])

        # add offset_mapping tokens to offset_maps
        offset_mapping_chunks[i] = torch.cat([
            torch.tensor([[0,0]]), offset_mapping_chunks[i], torch.tensor([[0,0]])
        ])
        
    # Step4: get required padding length
    pad_len = chunksize - input_id_chunks[i].shape[0]
    # check if tensor length satisfies required chunk size
    if pad_len > 0:
        # if padding length is more than 0, we must add padding
        input_id_chunks[i] = torch.cat([
            input_id_chunks[i], torch.Tensor([0] * pad_len)
        ])
        mask_chunks[i] = torch.cat([
            mask_chunks[i], torch.Tensor([0] * pad_len)
        ])
        offset_mapping_chunks[i] = torch.cat([
            offset_mapping_chunks[i], torch.Tensor([[0,0]] * pad_len)
        ])
        
    # Step 5: Stack the chunks
    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(mask_chunks)
    offset_mapping_merged = torch.stack(offset_mapping_chunks)

    input_ids = input_ids.long()
    attention_mask = attention_mask.int()
    
    # Step6 : unsqueeze it
    input_ids = input_ids.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)
    offset_mapping_merged = offset_mapping_merged.unsqueeze(0)

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    offset_mapping_merged = offset_mapping_merged.to(device)
    
    # Step7: Call the model
    outputs = model(input_ids[0], attention_mask=attention_mask[0])
    tr_logits = outputs[0]
    tr_logits_mod = torch.argmax(tr_logits, axis=2)
    
    
    # Step8 - map sub_words to words
    MAX_LEN = 512
    tokens_words = [[-1] * MAX_LEN for i in range(tr_logits_mod.shape[0])]
    
    w_i = 0
    off = offset_mapping_merged[0]
    for itr1 in range(off.shape[0]):
        for itr, off_map in enumerate(off[itr1]):
            if off_map[0] == 0 and off_map[1] != 0:
                w_i += 1
                tokens_words[itr1][itr] = w_i
            elif off_map[0] != 0 and off_map[1] != 0:
                tokens_words[itr1][itr] = w_i
                
        
    return tr_logits_mod, tokens_words

In [8]:
def get_predictions(tr_logits_mod, tokens_words):
    
    # Step1 - reshape the logits and tokens_words
    tr_logits_merged = torch.reshape(tr_logits_mod, (1, -1))
    tokens_words_merged = []
    for tokens in tokens_words:
        tokens_words_merged.extend(tokens)
        
    MAX_LEN_MOD = len(tokens_words_merged)
    all_predictions = {}
    pred = tr_logits_merged[0]
    i = 0
    while i<MAX_LEN_MOD:
        prediction = []
        start = pred[i]
        if start in [0,2,4,6,8,10,12,14]:
            label = idx_to_label[start.item()]
            label_mod = label.replace("B-", "")
            prediction.append(tokens_words_merged[i])
            i += 1
            if i>=MAX_LEN_MOD: break
            while pred[i]==start+1:
                # if we have "I-" after "B-" 
                if not tokens_words_merged[i] in prediction:
                    prediction.append(tokens_words_merged[i])
                i += 1
                if i>=MAX_LEN_MOD: break
        else:
            i += 1
        prediction = [x for x in prediction if x!=-1]
        if len(prediction)>4:
            if label_mod not in all_predictions:
                all_predictions[label_mod] = [' '.join([str(x) for x in prediction])]
            else:
                all_predictions[label_mod].append(' '.join([str(x) for x in prediction]))
                
    return all_predictions

In [9]:
row_df = [] # list will have the rows for submission_df
folder = "../input/feedback-prize-2021/test"
for file_name in os.listdir(folder):
    if file_name.endswith(".txt"):
        with open(os.path.join(folder, file_name), "r") as fin:
            content = fin.read()
            
    # get the predictions from the model
    tr_logits_mod, tokens_words = model_pred(content)
    
    # convert the predictions into the desired format
    all_predictions = get_predictions(tr_logits_mod, tokens_words)
    
    # store the predicitons in the list
    for label in all_predictions:
        for pred_str in all_predictions[label]:
            ind_dct = {}
            ind_dct['id'] = file_name.replace(".txt", "")
            ind_dct['class'] = label
            ind_dct['predictionstring'] = pred_str
            row_df.append(ind_dct)

Token indices sequence length is longer than the specified maximum sequence length for this model (754 > 512). Running this sequence through the model will result in indexing errors


In [10]:
sub_df = pd.DataFrame(row_df)
sub_df.head()

,id,class,predictionstring
0,0FB0700DAF44,Lead,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,0FB0700DAF44,Position,42 43 44 45 46 47 48
2,0FB0700DAF44,Claim,50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
3,0FB0700DAF44,Claim,67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 8...
4,0FB0700DAF44,Claim,197 198 199 200 201 202 203 204 205 206 207 20...


In [11]:
# sub_df[sub_df['id'] == 'DF920E0A7337']

In [12]:
sub_df.to_csv("submission.csv", index=False)

In [13]:
sub_df.shape

(50, 3)